In [74]:
import pandas as pd
import re
import numpy as np
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Charger l'extraction Tabula

In [75]:
df = pd.read_csv('tabula-compenswiss - list of positions - 30.06.2022.csv')

## Nettoyer l'extraction Tabula
J'ai gardé les entêtes de chaque page pour garder les bonnes colonnes lors de l'extraction. On a plus besoin de ces entêtes.

In [77]:
df = df[df['ISIN'] != 'ISIN']

Formate les nombres

In [78]:
# Supprime les séparateurs de milliers
df['Market Value CHF'] = df['Market Value CHF'].replace("'",'', regex=True)

# Supprime les décimales
df['Market Value CHF'] = df['Market Value CHF'].replace("\..*",'', regex=True)

# Convertit les montants en entiers
df['Market Value CHF'] = df['Market Value CHF'].astype(int)

<ipython-input-78-6ce08ba86458>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Market Value CHF'] = df['Market Value CHF'].replace("'",'', regex=True)
<ipython-input-78-6ce08ba86458>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Market Value CHF'] = df['Market Value CHF'].replace("\..*",'', regex=True)
<ipython-input-78-6ce08ba86458>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [79]:
df.head()

,ISIN,Asset Class,Description,Nominal/Quantity,Market Value CHF
0,XS2459025909,Money market Investments,0 ALLZ 22 ECP,EUR 3'000'000,3008851
1,CH0496692960,Money market Investments,0 BALOISE HLDG 22,CHF 8'100'000,8108100
2,CH0496692960,Money market Investments,0 BALOISE HLDG 22,CHF 8'400'000,8392440
3,CH0496692960,Money market Investments,0 BALOISE HLDG 22,CHF 1'000'000,999100
4,CH0407153342,Money market Investments,0 BLKB 23,CHF 5'000'000,4972500


## Vérifier les montants

In [80]:
df['Market Value CHF'].sum()

34454523303

In [81]:
len(df)

5514

## Mapping Bloomberg pour identifier les compagnies et leur catégorie d'activité


In [89]:
### Charger le fichier Bloomberg (Finanz & Wirtschaft)
### Créer un dictionnaire avec des ISIN uniques pour identifier l'entreprise

bloom = pd.read_excel('mapping.xlsx')
len(bloom)

20318

In [93]:
bloom['Date'].value_counts()

2022-06-30    3911
2021-12-31    1408
2021-06-30     879
2021-03-31     252
Name: Date, dtype: int64

In [90]:
bloom = bloom.drop_duplicates(subset=['ISIN'])

In [91]:
len(bloom)

6450

In [92]:
bloom['Date'].value_counts()

2022-06-30    3911
2021-12-31    1408
2021-06-30     879
2021-03-31     252
Name: Date, dtype: int64

In [94]:
bloom.sample()

,ISIN,Date,Name,Ticker,Wertpapiertyp,Emittent,Company Name,Vollständiger Name,Kurzname,Gruppe,Sektor,Untergruppe,Country (P),Country ISO,Land (Risiko),Sitzland
5705,XS1806130305,2021-12-31,CKHH 2 04/13/30,CKHH,Corporate Bonds,CK HUTCHISON EUR FIN 18,CK Hutchison Europe Finance 18 Ltd,CKHH 2 04/13/30,CK HUTCHISON E F,Retail,"Consumer, Cyclical",Retail-Drug Store,United Kingdom,GB,GB,KY


In [83]:
df = pd.merge(bloom, df, on = "ISIN", how = "right", indicator = True)

# 234 lignes du fichier Compenswiss ne peuvent pas être identifiées
df['_merge'].value_counts()

both          5280
right_only     234
left_only        0
Name: _merge, dtype: int64

In [ ]:
# Exemple

df[df['ISIN'] == 'CA01626P3043']

In [ ]:
df[df['_merge'] == 'right_only'].head()

## Tests for manual update

In [ ]:
# Méthode pour combler les lacunes d'un dataframe avec un dataframe à la structure similaire

In [ ]:
df1 = pd.DataFrame({'id':[1,2],
                    'company': ['Novartis', np.NaN],
                    'category': [np.NaN, "NaN"]
                   })
df2 = pd.DataFrame({'id': [1,2],
                    'company': [np.NaN, 'Migros'],
                    'category': ["pharma", "retail"]
                   })

df1 = df1.set_index("id").combine_first(df2.set_index("id")).reset_index()
df1

## Editing

In [85]:
# Charger le fichier contenant des ajouts manuels faits sur Google Sheets

transfo = pd.DataFrame(pd.read_csv('transformations_manuelles.csv'))

In [95]:
transfo

,ISIN,Date,Name,Ticker,Wertpapiertyp,Emittent,Company Name,Vollständiger Name,Kurzname,Gruppe,Sektor,Untergruppe,Country (P),Country ISO,Land (Risiko),Sitzland,Asset Class,Description,Currency,Nominal/Quantity,Market Value CHF,Date.1,_merge
0,XS2429878700,NaN,NaN,NaN,NaN,NaN,OP YRITYSPANK,NaN,NaN,Banks,NaN,NaN,NaN,NaN,NaN,NaN,Money market Investments,0 OPCB 23 ECP,NaN,USD 2'000'000,1903345,True,right_only
1,XS2435086652,NaN,NaN,NaN,NaN,NaN,DNB Bank ASA,NaN,NaN,Banks,NaN,NaN,NaN,NaN,NaN,NaN,Money market Investments,0 DNBB 22 ECP,NaN,EUR 3'000'000,3015160,True,right_only
2,FR0127437160,NaN,NaN,NaN,NaN,NaN,Société Generale,NaN,NaN,Banks,NaN,NaN,NaN,NaN,NaN,NaN,Money market Investments,0 SG 23 ECP,NaN,USD 1'000'000,957350,True,right_only
3,XS2451802925,NaN,NaN,NaN,NaN,NaN,Svenska Handelsbanken AB,NaN,NaN,Banks,NaN,NaN,NaN,NaN,NaN,NaN,Money market Investments,0 SHB 22 ECP,NaN,EUR 3'000'000,3010201,True,right_only
4,US42225P5017,NaN,NaN,NaN,NaN,NaN,HEALTHCARE TRUST AME. HTA.,NaN,NaN,Engineering&Construction (medical office build...,NaN,NaN,NaN,NaN,NaN,NaN,Real Estate,HLTHC RLTY TR-A,NaN,150246,4014519,False,right_only
5,US276480AE09,NaN,NaN,NaN,NaN,NaN,Eastern Gas Transmission & Storage Inc,NaN,NaN,Gas,NaN,NaN,NaN,NaN,NaN,NaN,Foreign Currency bonds,4.8 EAST 43NTS-144A,NaN,USD 625'000,559122,False,right_only
6,FR0014009LO3,NaN,NaN,NaN,NaN,NaN,EssilorLuxottica S.A,NaN,NaN,Healthcare-Products,NaN,NaN,NaN,NaN,NaN,NaN,Equities,ESSILOR RTS 22,NaN,84465,0,True,right_only
7,DE000A3MQQ17,NaN,NaN,NaN,NaN,NaN,Fresenius SE & Co. KGaA,NaN,NaN,Healthcare-Products,NaN,NaN,NaN,NaN,NaN,NaN,Equities,FRESENI RTS.22,NaN,125822,0,True,right_only
8,CA01626P3043,NaN,NaN,NaN,NaN,NaN,Alimentation Couche-Tard Inc Share Price,NaN,NaN,Retail,NaN,NaN,NaN,NaN,NaN,NaN,Equities,ALIMENT COUCHE -A,NaN,43286,1613004,False,right_only
9,CA87971M9969,NaN,NaN,NaN,NaN,NaN,Telus,NaN,NaN,Telecommunications,NaN,NaN,NaN,NaN,NaN,NaN,Equities,TELUS NON-CDN,NaN,23068,490835,False,right_only


In [87]:
len(transfo)

10

In [ ]:
# Fill df NaNs with transfo values (Company Name and Gruppe), 10 rows modified
# Combler les cellules manquants des colonnes Company Name et Gruppe (NaN) du fichier Compenswiss avec les ajouts manuels

df = df.set_index("ISIN").combine_first(transfo.set_index("ISIN")).reset_index()


In [ ]:
# Exemples

df[df.apply(lambda row: row.astype(str).str.contains("US276480AE09", case=False).any(), axis=1)]


In [ ]:
df[df.apply(lambda row: row.astype(str).str.contains("XS2429878700", case=False).any(), axis=1)]


## Analyses

#### La valeur la plus haute est un fond Pictet

In [ ]:
df[df['Market Value CHF'] == df['Market Value CHF'].max()]

In [ ]:
df.nlargest(10, 'Market Value CHF')

#### Comptabiliser les montants investis par groupe d'activité

#### Oil & Gas

In [ ]:
oil = df[(df['Gruppe'] == 'Oil and gas') | 
         (df['Gruppe'] == 'Oil&Gas') | 
         (df['Gruppe'] == 'Oil&Gas Services') |
         (df['Gruppe'] == 'Gas') | 
         (df['Gruppe'] == 'Pipelines') 
        ]
        

oil[['Company Name', 'Gruppe', 'Market Value CHF']].head()

In [ ]:
oil['Market Value CHF'].sum()

#### Tobacco

In [ ]:
tobacco = df[df.apply(lambda row: row.astype(str).str.contains("tobacco", case=False).any(), axis=1)]

In [ ]:
tobacco['Market Value CHF'].sum()

In [ ]:
tobacco[['Company Name', 'Gruppe', 'Market Value CHF']]

#### Armes, militaire, défense

In [96]:
weapons = df[(df['Gruppe'] == 'Aerospace/Defense') | 
             (df['Untergruppe'] == 'Explosives')
            ]

len(weapons)

30

In [ ]:
weapons['Market Value CHF'].sum()

#### Nucléaire

In [ ]:
nuclear = df[df.apply(lambda row: row.astype(str).str.contains('nuclear', case=False).any(), axis=1)]
nuclear

In [ ]:
nuclear['Market Value CHF'].sum()

#### Voitures

In [ ]:
auto = df[(df['Gruppe'] == 'Auto Parts') | 
         (df['Gruppe'] == 'Auto ABS') | 
         (df['Gruppe'] == 'Auto Manufacturers') |
         (df['Gruppe'] == 'Auto Parts&Equipment') | 
         (df['Untergruppe'] == 'Retail-Automobile') | 
         (df['Untergruppe'] == 'Retail-Auto Parts') | 
         (df['Untergruppe'] == 'Rental Auto/Equipment')
        ]
auto['Market Value CHF'].sum()


#### Mining

In [ ]:
mining = df[(df['Gruppe'] == 'Mining')| 
         (df['Untergruppe'] == 'Machinery-Constr&Mining')
           ]



mining[['Company Name', 'Gruppe', 'Untergruppe', 'Market Value CHF']].groupby('Company Name')['Market Value CHF'].sum().to_clipboard()

In [ ]:
mining['Market Value CHF'].sum()

In [ ]:
mining['Company Name'].unique()

#### Total Energies

In [ ]:
total = df[df.apply(lambda row: row.astype(str).str.contains('total', case=False).any(), axis=1)]

total['Company Name'].value_counts()

In [ ]:
total['Market Value CHF'].sum()

#### Amazon

In [ ]:
amazon = df[df.apply(lambda row: row.astype(str).str.contains('amazon', case=False).any(), axis=1)]

amazon['Company Name'].value_counts()

In [ ]:
amazon['Market Value CHF'].sum()

#### BHP

In [ ]:
bhp = df[df.apply(lambda row: row.astype(str).str.contains('BHP', case=False).any(), axis=1)]

In [ ]:
bhp['Market Value CHF'].sum()

#### Energies renouvelables

In [ ]:
en = df[df['Gruppe'] == 'Energy-Alternate Sources']

en['Market Value CHF'].sum()

In [ ]:
en[['Company Name', 'Gruppe', 'Market Value CHF']]

#### Viande

In [ ]:
viande = df[df['Untergruppe'] == 'Food-Meat Products']
viande[['Company Name', 'Gruppe', 'Market Value CHF']]

In [ ]:
viande['Market Value CHF'].sum()

#### Eau

In [ ]:
df[df.apply(lambda row: row.astype(str).str.contains('water', case=False).any(), axis=1)]['Gruppe'].value_counts()

In [ ]:
eau = df[df['Gruppe'] == 'Water']


In [ ]:
eau['Market Value CHF'].sum()

In [ ]:
eau

In [ ]:
eau['Market Value CHF'].sum()

#### Papier

In [ ]:
papier = df[df['Gruppe'] == 'Forest Products&Paper']

In [ ]:
papier['Market Value CHF'].sum()

#### Air travel


In [ ]:
df[(df['Gruppe'] == 'Airlines') | (df['Untergruppe'] == 'Airport Develop/Maint')][['Company Name', 'Gruppe', 'Untergruppe', 'Market Value CHF']].groupby('Company Name')['Market Value CHF'].sum()

In [ ]:
df[(df['Gruppe'] == 'Airlines') | (df['Untergruppe'] == 'Airport Develop/Maint')]['Market Value CHF'].sum()

In [ ]:
df[(df['Gruppe'] == 'Airlines') | (df['Untergruppe'] == 'Airport Develop/Maint')].groupby('Company Name')['Market Value CHF'].sum().to_clipboard()

#### Entreprises polémiques

In [ ]:
target = [
"Amazon",
"TotalEnergies",
"Toyota",
"Engie",
"Air Liquide",
"Enel",
"Iberdrola",
"Shell",
"Exxon",
"Gulf Power",
"Vinci",
"Var Energi",
"Monongahela Power Company - Mon Power",
"Monongahela",
"Philip Morris PMI",
"PMI",
"Philip Morris",
"Airbus",
"Flughafen Zurich",
"ZRH",
"Safran",
"Dassault",
"SIKA",
"Holcim",
"Danone",
"Shin-Etsu Chemical",
"Shin-Etsu",
"Pepsi",
"Bayer",
"Pernod-Ricard",
"Diageo",
"Lonza",
"Henkel",
"Walmart",
"BASF",
"Nestlé",
"Coca Cola",
"Unilever",
"Procter and Gamble",
"Procter & Gamble",
"Procter&Gamble",
"Mercedes",
"Tesla",
"Mitsubishi",
"Schneider Electric",
"ABB",
"LVMH",
"Siemens",
"Stellantis",
"Huntington Ingalls Industries Inc",
"Huntington Ingalls",
"BHP Ltd",
"EDF",
"Arcelor Mittal",
"Air Canada",
"Continental Airlines",
"Cimic group",
"Ballard Systems Power",
"Atco ltd/Canada",
"Ivanhoe Mines Ltd",
"Pan American Silver Corp",
"Lundin Mining Corp",
"Fresnillo PLC",
"Magellan Midstream Partner",
"Parkland Corp",
"Yamana Gold inc",
"Kinross Gold",
"Keyera corp",
"Delta Airlines",
"B2 Gold corp",
"AltaGas LTD",
"Heico Corp",
"SouthWest Airlines",
"Mercury NZ",
"Qantas Airways",
"Qantas",
"Diamondback Energy inc",
"Evolution mining ltd",
"Air China",
"Coal india",
"Indian oil",
"Petronas",
"Polish oil and gas company",
"Ugi corp",
"Imperial oil ltd"]

In [ ]:
df_target = df[df.apply(lambda row: row.astype(str).str.contains('|'.join(target), case=False).any(), axis=1)]
df_target.groupby('Company Name')['Market Value CHF'].sum().to_clipboard()

#### Check environnemental

In [ ]:
env = ['Albioma',
 'Alpha Metallurgical',
 'Ameren',
 'American Electric Power',
 'Arch Resources',
 'Athabasca',
 'BP p.l.c.',
 'British Petroleum',
 'Berry Corp',
 'Buzzi Unicem',
 'CF Industries',
 'Canadian Natural Resources',
 'Chevron',
 'Coronado Global Resources',
 'Denbury',
 'Electric Power Development',
 'Eni S.p.A.',
 'Eni SpA',
 'Entergy Corp',
 'Equinor',
 'Exxon',
 'HeidelbergCement',
 'Hokkaido Electric Power',
 'Hokuriku Electric Power',
 'Holcim',
 'New Hope',
 'OCI N.V.',
 'OMV AG',
 'Repsol',
 'Shell',
 'Stanmore',
 'Sumitomo Osaka Cement',
 'Taiheiyo Cement',
 'TerraCom',
 'AES Corp/The',
 "AES Corp",
 'The Scotts Miracle-Gro',
 'ScottsMiracle-Gro',
 'Southern Company',
 'Titan',
 'TransAlta',
 'TransAlta',
 'Vicat',
 'Vistra',
 'Soul Pattinson',
 'WHSP',
 'Yara',
 'eREX']

In [ ]:
df_env = df[df.apply(lambda row: row.astype(str).str.contains('|'.join(env), case=False).any(), axis=1)]
df_env.groupby('Company Name')['Market Value CHF'].sum().to_clipboard()

In [ ]:
len(df_env)

In [ ]:
df_env['Company Name'].value_counts().sort_values(ascending = False)

#### Holcim

In [ ]:
holcim = df[df.apply(lambda row: row.astype(str).str.contains('Holcim', case=False).any(), axis=1)]

In [ ]:
holcim['Market Value CHF'].sum()

In [ ]:
holcim['Date'].value_counts()

In [ ]:
holcim

#### 4 entreprises sans vergogne

In [ ]:
select = ['repsol', 'yara', 'cf industries', 'lundin', 'Aker BP']

df[df.apply(lambda row: row.astype(str).str.contains('|'.join(select), case=False).any(), axis=1)]

In [ ]:
df[df.apply(lambda row: row.astype(str).str.contains('|'.join(select), case=False).any(), axis=1)].groupby('Company Name')['Market Value CHF'].sum()


In [ ]:
df[df.apply(lambda row: row.astype(str).str.contains('Aker BP', case=False).any(), axis=1)]['Market Value CHF'].sum()

#### Russie

In [ ]:
russia = df[df.apply(lambda row: row.astype(str).str.contains('russi', case=False).any(), axis=1)]
len(russia)

In [ ]:
russia

#### Gold

In [ ]:
df[df.apply(lambda row: row.astype(str).str.contains('Gold - LBMA Good Delivery Bars', case=False).any(), axis=1)]

In [ ]:
# Pour compléter ces montants, recherche dans le détail du fonds Pictet

In [ ]:
df['Date'].value_counts()